In [5]:
!pip uninstall -y google-cloud-datastore

Uninstalling google-cloud-datastore-1.9.0:
  Successfully uninstalled google-cloud-datastore-1.9.0


In [2]:
!pip install google-cloud-datastore

    100% |████████████████████████████████| 102kB 771kB/s a 0:00:01
    100% |████████████████████████████████| 71kB 1.5MB/s ta 0:00:011
    100% |████████████████████████████████| 81kB 1.4MB/s ta 0:00:011
    100% |████████████████████████████████| 102kB 1.6MB/s a 0:00:011
    100% |████████████████████████████████| 81kB 674kB/s ta 0:00:01
  Running setup.py bdist_wheel for googleapis-common-protos ... done
  Stored in directory: /Users/elmehdiaitbrahim/Library/Caches/pip/wheels/9e/3d/a2/1bec8bb7db80ab3216dbc33092bb7ccd0debfb8ba42b5668d5
Successfully built googleapis-common-protos


In [3]:
from google.cloud import datastore

In [4]:
datastore_client = datastore.Client()

DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started

In [63]:
data = {}
sub_titles = []
action_text = ''
full_sub_title = ''
with open('IRSA-Manual-Succinct-Without-Annex.txt', 'r') as txt:
    for line in txt:
        if line.startswith('TITRE'):
            num = line[6:7]
            title = line[8:]
            title_num = 'titre ' + num
            data[title_num] = title.strip().lower()
            full_title = title_num + ' ' + data[title_num]
            if len(sub_titles) > 0:
                prev_title = data['titre ' + str(int(num)-1)]
                full_prev_title = 'titre ' + str(int(num)-1) + ' ' + prev_title
                data[full_prev_title] = sub_titles
                sub_titles = []
            if action_text != '':
                data[full_sub_title] = action_text
                action_text = ''
        elif line.startswith(num):
            sub_num = line[:3]
            sub_title = line[4:]
            data[sub_num] = sub_title.strip().lower()
            full_sub_title = sub_num + ' ' + data[sub_num]
            sub_titles.append(full_sub_title)
            
            if action_text != '':
                arr = sub_num.split('-')
                second_num = int(arr[1]) - 1
                if second_num > 0:
                    prev_sub_num = arr[0] + '-' + str(second_num)
                    prev_sub_title = data[prev_sub_num]
                    full_prev_sub_title = prev_sub_num + ' ' + prev_sub_title
                    data[full_prev_sub_title] = action_text
                    action_text = ''
        elif line != '\r\n':
            action_text += line
    data[full_sub_title] = action_text    

{'titre 1': 'dispositions generales', '1-1': 'principe fondamental – indemnisation directe de l’assure', '1-2': 'champ d’application', '1-1 principe fondamental – indemnisation directe de l’assure': "\n\nQuels que soient la typologie de l'accident de la circulation, la nature et le montant des dommages, les sociétés adhérentes s'obligent, préalablement à l'exercice de leurs recours, à indemniser elles-mêmes leurs assurés, dans la mesure de leur droit à réparation, déterminé selon les règles du droit commun.\nL'assureur direct doit indemniser son assuré même si :\n• la réclamation est présentée pour son compte par un mandataire, une société de protection juridique, un assureur dommages etc,..\n• son contrat comporte une franchise de responsabilité quel que soit son montant.\nLes sociétés adhérentes s’interdisent de prendre l’initiative d’indemniser directement une\nvictime assurée auprès d’une société adhérente.\nElles s’obligent par ailleurs à transmettre 

In [ ]:
for key in data:    
    kind = 'Topic'
    topic_key = datastore_client.key(kind, key)

    topic = datastore.Entity(key=topic_key)
    topic['action_text'] = data[key]

    datastore_client.put(topic)
    print('Saved {}: {}'.format(topic.key.name, topic['action_text']))

In [7]:
print('Done processing IRSA doc.')

Done processing IRSA doc.
